In [1]:
import sys
from bs4 import BeautifulSoup
import requests
import datetime as dt
from PIL import Image
import helpers as hf

import yaml
from unidecode import unidecode
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont
import json
import numpy as np
from instagrapi import Client as instaClient
import tempfile
from PIL import Image
from io import BytesIO
with open("info/design_config.yml", "r") as f:
    design_config = yaml.safe_load(f)

design_config,shop_config = hf.get_config('cbb')
cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])

private file size: 2640


In [15]:
teams = cfbd_loader.get_team_info()

In [116]:
# check if printify ptoduct is posted
headers_printify = {
    "Authorization": f"Bearer {shop_config['printify_access']}",
    "Content-Type": "application/json",
}
headers_shopify = {
            "X-Shopify-Access-Token": shop_config["shopify_access"],
            "Content-Type": "application/json",
        }

In [117]:
products_link = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/products.json?limit=250"
response = requests.get(products_link, headers=headers_shopify)
response.headers["X-Shopify-Shop-Api-Call-Limit"]

'1/40'

In [3]:
ii_response = requests.get('https://cdn.shopify.com/s/files/1/0685/3509/1421/files/2333403980485870616_2048.jpg?v=1708887700')
img = Image.open(BytesIO(ii_response.content))

In [2]:
instagram = hf.instagrammer(
    un=shop_config["instagram"]["username"],
    pw=shop_config["instagram"]["password"],
    email=shop_config["instagram"]["eusername"],
    emailpw=shop_config["instagram"]["epassword"],
)

In [4]:
instagram.single_post(img,'test')

Attempting single image post


In [15]:
call_limit = response.headers["X-Shopify-Shop-Api-Call-Limit"]
call_split = [int(x) for x in call_limit.split('/')]
remaining_calls = call_split[1]-call_split[0]

In [10]:
print(len(prods))
print(len(prods2))

472
236


In [ ]:
import time

In [4]:
graphql_url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2023-04/graphql.json"


In [5]:
big_body ="""
            {
            product(id:"gid://shopify/Product/prod_id") {
                title
                media(first:5) {
                edges {
                    node {
                    ... fieldsForMediaTypes
                    }
                }
                }
            }
            }

            fragment fieldsForMediaTypes on Media {
            alt
            mediaContentType
            preview {
                image {
                id
                altText
                url
                }
            }
            status
            ... on Video {
                id
                sources {
                format
                height
                mimeType
                url
                width
                }
                originalSource {
                format
                height
                mimeType
                url
                width
                }
            }
            ... on ExternalVideo {
                id
                host
                embeddedUrl
            }
            ... on Model3d {
                sources {
                format
                mimeType
                url
                }
                originalSource {
                format
                mimeType
                url
                }
            }
            ... on MediaImage {
                id
                image {
                altText
                url
                }
            }
            }
            """

In [6]:
prod_id = 8368514007261

In [7]:
all_p = ify_user.recur_get_products()

In [9]:
for x in all_p:
    if x['id'] == prod_id:
        prod = x

In [12]:
prod_gql = prod["admin_graphql_api_id"]
body = big_body.replace("prod_id", str(prod_id))
# now that I have the images I can reorder them
url = f"https://{shop_config['cbb']['shop_name']}.myshopify.com/admin/api/2024-01/graphql.json"
# from this response i can get my images
response = requests.post(
    url=url, headers=headers_shopify, json={"query": body}
)

images = json.loads(response.text)["data"]["product"]["media"]["edges"]
image0 = images[0]["node"]["id"]
image2 = images[2]["node"]["id"]

In [14]:
title = prod['title']

In [15]:
alt_text_query = "mutation productUpdateMedia($media: [UpdateMediaInput!]!, $productId: ID!) { productUpdateMedia(media: $media, productId: $productId) { media { alt } } }"

alt_text_var = {
"media": [
    {
    "alt": title+' back view',
    "id": image0
    }
],
"productId": prod_gql
}

alt_resp = requests.post(
    graphql_url, headers=headers_shopify, json={"query": alt_text_query, "variables": alt_text_var}
)

In [ ]:
import numpy as np

In [16]:
'currentlyAvailable' in str(alt_resp.headers)

False

In [26]:
json.loads(alt_resp.text)['extensions']['cost']['throttleStatus']['currentlyAvailable'] < 100:

False

In [32]:
alt_resp.headers["extensions"]

KeyError: 'extensions'